Goal: create functions to mine data for aggregate stats

In [1]:
import pandas as pd

In [2]:
import sqlite3

In [3]:
con=sqlite3.connect('../players.db')

In [4]:
df = pd.read_sql('select * from results',con)

In [16]:
df

,index,Year,Match,Format,European_player,European_score,American_score,American_player,Europe_won
0,0,1994,0,Singles,Tom Storm,2,0,Bobby Hunter,1
1,1,1994,1,Singles,Oliver Ortmann,2,0,Mark Wilson,1
2,2,1994,2,Singles,Lee Tucker,1,2,Lou Butera,0
3,3,1994,3,Singles,Allison Fisher,0,2,Vivian Villarreal,0
4,4,1994,4,Singles,Steve Davis,2,1,Mike Gulyassy,1
...,...,...,...,...,...,...,...,...,...
506,506,2019,14,Doubles,"Niels Feijen, Jayson Shaw",5,4,"Justin Bergman, Billy Thorpe",1
507,507,2019,15,Singles,Jayson Shaw,3,5,Billy Thorpe,0
508,508,2019,16,Singles,Eklent Kaçi,5,1,Tyler Styer,1
509,509,2019,17,Singles,Alexander Kazakis,3,5,Shane Van Boening,0


In [5]:
import pickle
am_players = pickle.load(open('american_players','rb'))
euro_players = pickle.load(open('european_players','rb'))

In [6]:
d={}
for player in am_players:
    d[player] = df['American_player'].str.contains(player)
for player in euro_players:
    d[player] = df['European_player'].str.contains(player)

In [7]:
#Create shorthands for boolean indexing the dataframe
singles = df['Format']=='Singles'
doubles = df['Format']=='Doubles'
teams = df['Format']=='Teams'
triples = df['Format']=='Triples'
europe_won = df['Europe_won'] == True
america_won = df['Europe_won'] == False

In [8]:
def winlosslist(player, country, df=df):
    
    #singles
    if 'a' in country:
        swin=df[america_won&singles&d[player]].shape[0]
        sloss=df[d[player]&~america_won&singles].shape[0]
        smp=swin+sloss
    else:
        sloss=df[d[player]&america_won&singles].shape[0]
        swin=df[d[player]&~america_won&singles].shape[0]    
        smp=swin+sloss
    swl='{}-{}'.format(swin,sloss)
    try:
        spct=swin*100/(swin+sloss)
    except:
        spct=0
    
    #doubles
    if 'a' in country:
        dwin=df[d[player]&america_won&doubles].shape[0]
        dloss=df[d[player]&~america_won&doubles].shape[0]
        dmp=dwin+dloss
    else:
        dloss=df[d[player]&america_won&doubles].shape[0]
        dwin=df[d[player]&~america_won&doubles].shape[0] 
        dmp=dwin+dloss
    dwl='{}-{}'.format(dwin,dloss)
    try:
        dpct=dwin*100/(dwin+dloss)   
    except:
        dpct=0
    
        #teams
    if 'a' in country:
        twin=df[d[player]&america_won&teams].shape[0]
        tloss=df[d[player]&~america_won&teams].shape[0]
        tmp=twin+tloss
    else:
        tloss=df[d[player]&america_won&teams].shape[0]
        twin=df[d[player]&~america_won&teams].shape[0] 
        tmp=twin+tloss
    twl='{}-{}'.format(twin,tloss)
    try:
        tpct=twin*100/(twin+tloss)   
    except:
        tpct=0
        
        #triples
    if 'a' in country:
        trwin=df[d[player]&america_won&triples].shape[0]
        trloss=df[d[player]&~america_won&triples].shape[0]
        trmp=trwin+trloss
    else:
        trloss=df[d[player]&america_won&triples].shape[0]
        trwin=df[d[player]&~america_won&triples].shape[0] 
        trmp=trwin+trloss
    trwl='{}-{}'.format(trwin,trloss)
    try:
        trpct=round(trwin*100/(trwin+trloss),2)        
    except:
        trpct=0

    #combined
    cwin=dwin+swin+twin+trwin
    closs=dloss+sloss+tloss+trloss
    cmp=cwin+closs
    cwl='{}-{}'.format(cwin,closs)
    cpct=cwin*100/(cwin+closs)
    l=[player,country,smp,swin,sloss,swl,round(spct,2),dmp,dwin,dloss,dwl,round(dpct,2), twl,trwl,cmp, cwin,closs,cwl,round(cpct,2)]
    return(l)

In [9]:
#prepare list of winlosslists for each player to create player stats dataframe
L=[]
for player in am_players:
    try:
        lst = winlosslist(player,'America')
        L.append(lst)
    except: #Zerodivision error:
        print('failure- am: ', player)
#         pass
for player in euro_players:
    try:
        lst = winlosslist(player,'Europe')
        L.append(lst)
    except: #Zerodivision error:
        print('failure -eu ', player)
#         pass

In [10]:
#create player_stats dataframe
colz = 'Player,Team,Smp,Swin,Sloss,Swl,Spct,Dmp,Dwin,Dloss,Dwl,Dpct,Twl,Trwl,Cmp,Cwin,Closs,Cwl,Cpct'.split(',')
player_stats = pd.DataFrame(L, columns = colz)

In [11]:
player_stats[player_stats.Player=='Skyler Woodward']

,Player,Team,Smp,Swin,Sloss,Swl,Spct,Dmp,Dwin,Dloss,Dwl,Dpct,Twl,Trwl,Cmp,Cwin,Closs,Cwl,Cpct
19,Skyler Woodward,America,9,5,4,5-4,55.56,16,8,8,8-8,50.0,1-3,0-0,29,14,15,14-15,48.28


In [12]:
player_stats[player_stats.Player=='Tyler Styer']

,Player,Team,Smp,Swin,Sloss,Swl,Spct,Dmp,Dwin,Dloss,Dwl,Dpct,Twl,Trwl,Cmp,Cwin,Closs,Cwl,Cpct
15,Tyler Styer,America,4,1,3,1-3,25.0,5,4,1,4-1,80.0,0-1,0-0,10,5,5,5-5,50.0


In [15]:
player_stats[player_stats.Player=='Albin Ouschan']

,Player,Team,Smp,Swin,Sloss,Swl,Spct,Dmp,Dwin,Dloss,Dwl,Dpct,Twl,Trwl,Cmp,Cwin,Closs,Cwl,Cpct
70,Albin Ouschan,Europe,5,4,1,4-1,80.0,9,7,2,7-2,77.78,2-1,0-0,17,13,4,13-4,76.47


In [17]:
player_stats[player_stats.Player=='Joshua Filler']

,Player,Team,Smp,Swin,Sloss,Swl,Spct,Dmp,Dwin,Dloss,Dwl,Dpct,Twl,Trwl,Cmp,Cwin,Closs,Cwl,Cpct
58,Joshua Filler,Europe,6,5,1,5-1,83.33,4,2,2,2-2,50.0,1-0,0-0,11,8,3,8-3,72.73


In [13]:
player_stats

,Player,Team,Smp,Swin,Sloss,Swl,Spct,Dmp,Dwin,Dloss,Dwl,Dpct,Twl,Trwl,Cmp,Cwin,Closs,Cwl,Cpct
0,John DiToro,America,3,2,1,2-1,66.67,2,1,1,1-1,50.00,0-0,0-0,5,3,2,3-2,60.00
1,Charlie Williams,America,8,6,2,6-2,75.00,13,6,7,6-7,46.15,0-0,0-0,21,12,9,12-9,57.14
2,Reed Pierce,America,4,4,0,4-0,100.00,6,1,5,1-5,16.67,0-0,0-0,10,5,5,5-5,50.00
3,Corey Deuel,America,15,7,8,7-8,46.67,27,14,13,14-13,51.85,5-3,2-0,52,28,24,28-24,53.85
4,Mike Davis,America,2,1,1,1-1,50.00,2,1,1,1-1,50.00,2-0,1-1,8,5,3,5-3,62.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,Allison Fisher,Europe,1,0,1,0-1,0.00,2,0,2,0-2,0.00,0-0,0-0,3,0,3,0-3,0.00
78,Mika Immonen,Europe,33,16,17,16-17,48.48,39,22,17,22-17,56.41,3-3,1-1,80,42,38,42-38,52.50
79,Steve Knight,Europe,7,4,3,4-3,57.14,11,4,7,4-7,36.36,0-0,0-0,18,8,10,8-10,44.44
80,Vincent Facquet,Europe,2,0,2,0-2,0.00,1,1,0,1-0,100.00,0-0,0-0,3,1,2,1-2,33.33


In [18]:
player_stats.to_sql('playerstats',con,if_exists='replace')